### NLP Assignment-4

Load data

In [ ]:
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/train_split.csv
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/test_split.csv

--2024-11-15 18:24:52--  https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/train_split.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 144474 (141K) [text/plain]
Saving to: ‘train_split.csv’

train_split.csv     100%[===================>] 141.09K  --.-KB/s    in 0.03s   

2024-11-15 18:24:52 (5.09 MB/s) - ‘train_split.csv’ saved [144474/144474]

--2024-11-15 18:24:53--  https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/test_split.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35259 (34K) 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

In [ ]:
!pip install nltk

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertModel, BertTokenizer, RobertaModel, RobertaTokenizer
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score
import pandas as pd
import shutil
import sys
from transformers import BertConfig, BertForSequenceClassification
from transformers import RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer
import numpy as np
import torch.nn.functional as F
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [ ]:
data = pd.read_csv('train_split.csv')
data.head()

,text,Joy,Fear,Anger,Sadness,Surprise
0,The light would come all the way up to your ve...,0,1,0,0,1
1,"Well, my birthday is in January.",0,0,0,0,0
2,"Tears in my eyes, too much on my mind, and dee...",0,1,0,1,0
3,"My eyes scanned quickly into every room, every...",0,1,1,0,0
4,I felt the afterglow of the late afternoon sun...,1,0,0,0,0


In [ ]:
val_df = pd.read_csv('test_split.csv')
val_df.head()

,text,Joy,Fear,Anger,Sadness,Surprise
0,And they probably knew exactly what your dad saw.,0,1,0,0,1
1,"My neck still hurts while I'm working, and I w...",0,1,1,0,0
2,"and Christian fish bumper stickers, it caught ...",1,0,0,0,1
3,It felt so unreal...,0,1,0,0,1
4,' To be honest I don't know why I set out into...,0,1,0,0,0


In [ ]:
#hyperparameters

max_len =  256
train_batch_size = 4
valid_batch_size = 4
batch_size = 8
epochs = 5
LEARNING_RATE = 1e-5

In [ ]:
tokenizer =  RobertaTokenizer.from_pretrained('roberta-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, dataFrame, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataFrame
        self.text = dataFrame['text']
        self.max_len = max_len
        self.targets = self.data[dataFrame.columns[1:]]

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs['token_type_ids'].flatten(),
            'targets': torch.FloatTensor(self.targets.iloc[index].values)
        }

In [ ]:
val_df = pd.read_csv('test_split.csv')

In [ ]:
train_dataset = CustomDataset(data, tokenizer, max_len)
valid_dataset = CustomDataset(val_df, tokenizer, max_len)

In [ ]:
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size = train_batch_size, num_workers = 4, pin_memory=True, shuffle = True)
valid_data_loader = torch.utils.data.DataLoader(valid_dataset, batch_size = valid_batch_size, num_workers = 4, pin_memory=True, shuffle = False)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    '''torch.save(state, f_path)
    # if it is best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)'''

    torch.save(model.state_dict(), f_path)
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [ ]:
class RoBERTaClass(torch.nn.Module):
    def __init__(self):
        super(RoBERTaClass, self).__init__()
        # Initialize the RoBERTa model
        self.roberta_model = RobertaModel.from_pretrained('roberta-base')  # Use the appropriate model
        self.dropout = torch.nn.Dropout(p=0.3)  # Dropout layer (optional)
        self.linear = torch.nn.Linear(768, 5)  # Linear layer for classification (assuming 5 classes)

    def forward(self, input_ids, attn_mask, token_type_ids=None):  # RoBERTa does not need token_type_ids
        output = self.roberta_model(input_ids, attention_mask=attn_mask)
        cls_output = output.last_hidden_state[:, 0, :]  # Use the [CLS] token
        output_dropout = self.dropout(cls_output)
        output = self.linear(output_dropout)
        return output

# Initialize the model and move to device (e.g., GPU if available)
model = RoBERTaClass()
model.to(device)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RoBERTaClass(
  (roberta_model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNor

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params = model.parameters(), lr = LEARNING_RATE)

In [ ]:
val_target = []
val_output = []

In [ ]:
def train_model(n_epochs, training_loader, validation_loader, model, optimizer, checkpoint_path, best_model_path):
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf

    for epoch in range(1, n_epochs + 1):
        train_loss = 0
        valid_loss = 0
        model.train()
        print('############# Epoch {}: Training Start   #############'.format(epoch))
        for batch_idx, data in enumerate(training_loader):
            #print('yyy epoch',batch_idx)
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)
            #loss = loss_fn(outputs, targets)

            optimizer.zero_grad()
            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))

        print('############# Epoch {}: Training End   #############'.format(epoch))

    print('############# Epoch {}: Validation Start   #############'.format(epoch))

    model.eval()
    with torch.no_grad():
        for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_target.extend(targets.cpu().detach().numpy().tolist())
            val_output.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
            #print('zzz epoch',batch_idx)

            # print training/validation statistics
        # calculate average loss over an epoch
        train_loss = train_loss / len(training_loader)
        valid_loss = valid_loss / len(validation_loader)
        print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
        ))

        # save checkpoints variables and add important data
        checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }
        # save checkpoint
        save_ckp(checkpoint, False, checkpoint_path, best_model_path)

        # if validation loss decreased, save the model
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min, valid_loss))
            # save checkpoint as best model
            save_ckp(checkpoint, True, checkpoint_path, best_model_path)
            valid_loss_min = valid_loss

    print('############# Epoch {}: Validation End   #############'.format(epoch))

    return model

In [ ]:
ckpt_path = '/content/drive/MyDrive/curr_ckpt_roberta'
best_model_path = '/content/drive/MyDrive/best_model_roberta.pt'

In [ ]:
#trained_model = train_model(epochs, train_data_loader, valid_data_loader, model, optimizer, ckpt_path, best_model_path)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics.cluster import contingency_matrix
from sklearn.metrics import classification_report

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

def validation():
    model.eval()
    val_target = []
    val_output = []
    with torch.no_grad():
        for batch_idx, data in enumerate(valid_data_loader, 0):
            ids = data['input_ids'].to(device, dtype=torch.long)
            mask = data['attention_mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.float)

            outputs = model(ids, mask, token_type_ids)

            # Extend target and output lists
            val_target.extend(targets.cpu().detach().numpy().tolist())
            val_output.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    # Convert probabilities to binary predictions
    val_target = np.array(val_target)
    val_output = (np.array(val_output) >= 0.5).astype(int)

    # Generate classification report
    print(
        classification_report(
            val_target, val_output,
            target_names=['Joy', 'Sadness', 'Fear', 'Anger', 'Surprise'],
            zero_division=0
        )
    )

# Adjust DataLoader
valid_data_loader = DataLoader(valid_dataset, batch_size=valid_batch_size, num_workers=2, shuffle=False)

model.load_state_dict(torch.load('/content/drive/MyDrive/best_model_roberta.pt', map_location=device))

# Validate the model
validation()


<ipython-input-23-4ba1d6c337f5>:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/best_model_roberta.pt', map_locati

              precision    recall  f1-score   support

         Joy       0.75      0.69      0.72        94
     Sadness       0.89      0.72      0.79       232
        Fear       0.70      0.31      0.43        52
       Anger       0.67      0.71      0.69       126
    Surprise       0.73      0.67      0.70       124

   micro avg       0.77      0.67      0.72       628
   macro avg       0.75      0.62      0.66       628
weighted avg       0.78      0.67      0.71       628
 samples avg       0.67      0.62      0.62       628



In [ ]:
import pandas as pd

# Load the training DataFrame
df = pd.read_csv('/content/train_split.csv')
print(df.head())

                                                text  Joy  Fear  Anger  \
0  The light would come all the way up to your ve...    0     1      0   
1                   Well, my birthday is in January.    0     0      0   
2  Tears in my eyes, too much on my mind, and dee...    0     1      0   
3  My eyes scanned quickly into every room, every...    0     1      1   
4  I felt the afterglow of the late afternoon sun...    1     0      0   

   Sadness  Surprise  
0        0         1  
1        0         0  
2        1         0  
3        0         0  
4        0         0  


###Evaluation
An example of evaluation:

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Example of gold labels (y_true) and predictions (y_pred)
y_true = np.array([[1, 0, 0, 1, 0],  # joy, sadness, fear, anger, surprise
                   [0, 1, 1, 0, 0],
                   [1, 0, 1, 0, 1]])

y_pred = np.array([[1, 0, 0, 1, 0],  # joy, sadness, fear, anger, surprise
                   [0, 1, 0, 1, 0],
                   [1, 0, 1, 0, 0]])

# Generate the classification report
report = classification_report(y_true, y_pred, target_names=['joy', 'sadness', 'fear', 'anger', 'surprise'], zero_division=0)

print(report)

              precision    recall  f1-score   support

         joy       1.00      1.00      1.00         2
     sadness       1.00      1.00      1.00         1
        fear       1.00      0.50      0.67         2
       anger       0.50      1.00      0.67         1
    surprise       0.00      0.00      0.00         1

   micro avg       0.83      0.71      0.77         7
   macro avg       0.70      0.70      0.67         7
weighted avg       0.79      0.71      0.71         7
 samples avg       0.83      0.72      0.77         7

